# **Modeling - Parisian Rents**

As we did the Exploratory Data Analysis in the previous notebook, let's work on the model that will try to predict the Paris rent of a location based on the features. 

In [1]:
import pandas as pd
import numpy as np
import random
import pickle

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold

from sklearn.linear_model import LinearRegression
linreg = LinearRegression()
from sklearn.feature_extraction import DictVectorizer
dv = DictVectorizer()
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
import xgboost as xgb

from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from hyperopt.pyll import scope

## **Feature Preprocessing**

In [3]:
# Get the data
df = pd.read_csv("data/logement-encadrement-des-loyers.csv", sep=";")
df_set = df.copy()[['Secteurs géographiques', 
                    'Numéro du quartier', 
                    'Nombre de pièces principales', 
                    'Epoque de construction',
                    'Type de location', 
                    'Loyers de référence']]

In [4]:
df.columns

Index(['Secteurs géographiques', 'Numéro du quartier', 'Nom du quartier',
       'Nombre de pièces principales', 'Epoque de construction',
       'Type de location', 'Loyers de référence',
       'Loyers de référence majorés', 'Loyers de référence minorés', 'Année',
       'Ville', 'Numéro INSEE du quartier'],
      dtype='object')

In [ ]:
# Preprocessing function
def preprocess(df: pd.DataFrame, dv: DictVectorizer, fit_dv: bool = False):
    ''' Create feature value vectors and one hot encode categorical
        columns.

        Args :
          - df : A Dataframe
          - dv : A DictVectorizer object (from Scikit Learn)
          - fit_dv : True or False, if True, it fits the dv with 
                     the specified Dataframe.
    '''
    categorical = ['Epoque de construction', 'Type de location']
    numerical = ['Secteurs géographiques', 'Numéro du quartier', 
                  'Nombre de pièces principales']
    dicts = df[categorical + numerical].to_dict(orient='records')
    if fit_dv:
        X = dv.fit_transform(dicts)
    else:
        X = dv.transform(dicts)
    return X, dv

In [ ]:
# Split train and test datasets
df_train, df_test = train_test_split(df_set, test_size=0.20, random_state=42)

In [ ]:
# Preprocess the data, split Inputs and Outputs
target = 'Loyers de référence'

# Inputs and DictVectorizer
X, dv = preprocess(df = df_train, dv = dv, fit_dv = True)
# Ouputs
y = df_train[target].values

In [ ]:
# Let's see the features that the dv chose
dv.get_feature_names_out()

array(['Epoque de construction=1946-1970',
       'Epoque de construction=1971-1990',
       'Epoque de construction=Apres 1990',
       'Epoque de construction=Avant 1946',
       'Nombre de pièces principales', 'Numéro du quartier',
       'Secteurs géographiques', 'Type de location=meublé',
       'Type de location=non meublé'], dtype=object)

In [ ]:
# Save the Dictvectorizer to use it in preprocessing during production
# Save the model with pickle
filename = 'preprocessor.b'
pickle.dump(dv, open(filename, 'wb'))

##  **The Baseline : Linear Regression**

I chose a simple Linear Regression to be my baseline. The idea will be to beat the score of the Linear Regression Model I will build. 

In [ ]:
# A cross validation on the train dataset
cv_results = cross_validate(linreg, X.toarray(), y)
print(f"Results of the CV : {[i for i in cv_results['test_score']]}")

Results of the CV : [0.8120212116199752, 0.807290992021981, 0.793182170945342, 0.804447990350114, 0.8113913808114921]


In [ ]:
# Fit the linear regression
linreg.fit(X, y)

LinearRegression()

In [ ]:
# Define our test dataset from the preprocess function
target = 'Loyers de référence'

X_test, _ = preprocess(df = df_test, dv = dv, fit_dv = False)
y_test = df_test[target].values

In [ ]:
# Get the score on the test dataset
linreg.score(X_test, y_test)

0.8131377842672293

In [ ]:
# Get RMSE of our test data
y_preds = linreg.predict(X_test)
mean_squared_error(y_test, y_preds, squared=False)

1.7518591698739399

We have now our **baseline**, the score that we have to beat is **81%** of accuracy and a RMSE of **1.736**.

## **Multiple Algorithms Implementation (Lasso, RandomForest, SVC, XGboost)**

Now that we know the baseline to beat, let's create a cross validation on some algorithms and see which one performs the best. After that, we will have to find the best hyperparameters.

In [ ]:
# Compare each model with a Cross Validation of 3 splits
def find_best_model_CV(X, y):
  ''' Do a cross validation on a models dictionnary.

      Args:
        - X : inputs
        - y : outputs (target predicted)
      
      Returns :
        It returns the results of the CV : the RMSE and
        the STD of the scoring value.
  '''
  
  models = {'LinearRegression' : linear_model.Lasso(), 'RandomForestRegressor' : RandomForestRegressor(max_depth=2, random_state=0), 
            'PolynomialSVR' : SVR(kernel='poly'), 'XGBRegressor' : xgb.XGBRegressor(objective ='reg:squarederror')}
  kfold = KFold(n_splits=3)
  for model_name, model in models.items():
    results2 = cross_validate(model, X, y, cv=kfold, scoring = 'neg_root_mean_squared_error')
    print(f"{model_name} - RMSE : {(results2['test_score'].mean())*-1} and STD {np.std(results2['test_score'])}")

In [ ]:
# Print each result for training data
find_best_model_CV(X, y)

LinearRegression - RMSE : 2.8091079774318497 and STD 0.007165816625124331
RandomForestRegressor - RMSE : 2.7766766358399195 and STD 0.03670564715077652
PolynomialSVR - RMSE : 2.9380560541186127 and STD 0.009547957753102626
XGBRegressor - RMSE : 0.9745685821364202 and STD 0.0036840826782667474


XGB Regressor is clearly the best model among those we have chosen.

In [ ]:
# Measure Generalization of the best model
xgb_model = xgb.XGBRegressor(objective ='reg:squarederror')
xgb_model.fit(X, y)

xgb_model.score(X_test, y_test)

0.9439643247485657

In [ ]:
# Get predictions and measure accuracy on true outputs
y_preds = xgb_model.predict(X_test)
mean_squared_error(y_test, y_preds, squared=False)

0.9593360141649778

It gets a great accuracy score and a RMSE under 1.

## Hyperparameters Optimization

We have to find the best value of the hyperparameters of our model.

In [ ]:
# Define an objective function with a loss function
# that hyperopt will minimize
def objective(params):
      ''' An objective function that declare a model
          from a set of hyperparameters, fit it on 
          the training data and compute the rmse with
          real outputs/predicted outputs.

          Args:
            - params : dict of hyperparameters
          
          Returns:
            A dictionnary of some variables such as the
            loss, the model...
      '''
      model = xgb.XGBRegressor(**params)
      model.fit(X, y)
      y_preds = model.predict(X_test)
      rmse = mean_squared_error(y_test, y_preds, squared=False)

      return {'loss': rmse, 'status': STATUS_OK, 'model': model}

# A search space of hyperparameters that the 
# hyperopt min function will use to find the 
# best set of hyperparameters
search_space = {
    'objective' : 'reg:squarederror',
    'max_depth': scope.int(hp.quniform('max_depth', 1, 20, 1)),
    'eta' : scope.int(hp.quniform('eta', 0, 1, 0.1)),
    'n_estimators': scope.int(hp.quniform('n_estimators', 10, 50, 1)),
    'min_samples_split': scope.int(hp.quniform('min_samples_split', 2, 10, 1)),
    'min_samples_leaf': scope.int(hp.quniform('min_samples_leaf', 1, 4, 1)),
    'random_state': 42
}

# Initialize an empty trials database used to stored
# data from the hyperopt process
trials = Trials()

# For reproducible results
rstate = np.random.RandomState(42)

# Get the best set of hyperparameters
best = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=trials,
    rstate=rstate
)

# Save and reload results
pickle.dump(trials, open("xgbregressor_trials.p", "wb"))
trials = pickle.load(open("xgbregressor_trials.p", "rb"))

print(best)

100%|██████████| 50/50 [00:12<00:00,  4.01it/s, best loss: 0.5148224655184348]
{'eta': 0.4, 'max_depth': 11.0, 'min_samples_leaf': 1.0, 'min_samples_split': 3.0, 'n_estimators': 50.0}


In [ ]:
# Save the best model from trials
best_model = trials.best_trial['result']['model']

In [ ]:
# Fit the best model with our training data
best_model.fit(X, y)

# Let's see the generalization score on our
# test dataset
best_model.score(X_test, y_test)

0.983862431714724

The model is rendenring great on test data, that is a good indication about the generalization of it on new data.

In [ ]:
best_model.save_model("final_xgbregressor_model.model")

In [ ]:
# Save the model with pickle
filename = 'final_xgbregressor_model.pkl'
pickle.dump(best_model, open(filename, 'wb'))

Let's deploy the model on a web application !